In [2]:
import sys, os
import numpy as np
import time
from sklearn import decomposition
import pandas as pd
import torch
import PIL.Image

print(sys.path)

# root directory is 2 dirs up from this file
root = '/usr/local/serenceslab/maggie/shapeDim/'


feat_dir = os.path.join(root, 'Analysis', 'image_similarity', 'features')
image_dir = os.path.join(root, 'Stimuli','AmpGrid3_adj_full_grey_small')


['/mnt/neurocube/local/serenceslab/maggie/shapeDim/Analysis/image_similarity', '/usr/local/serenceslab/maggie/conda_envs/shape_dim/lib/python37.zip', '/usr/local/serenceslab/maggie/conda_envs/shape_dim/lib/python3.7', '/usr/local/serenceslab/maggie/conda_envs/shape_dim/lib/python3.7/lib-dynload', '', '/usr/local/serenceslab/maggie/conda_envs/shape_dim/lib/python3.7/site-packages', '/usr/local/serenceslab/maggie/conda_envs/shape_dim/lib/python3.7/site-packages/IPython/extensions', '/mnt/neurocube/home/mmhender/.ipython']


/usr/local/serenceslab/maggie/conda_envs/shape_dim/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
import h5py

fn2save = os.path.join(feat_dir,'Images_grid3_all.h5py')

with h5py.File(fn2save, 'r') as f:
    
    dtmp = np.array(f['/stimuli'])

In [10]:
dtmp[200,0,:,:]

array([[77, 77, 77, ..., 77, 77, 77],
       [77, 77, 77, ..., 77, 77, 77],
       [77, 77, 77, ..., 77, 77, 77],
       ...,
       [77, 77, 77, ..., 77, 77, 77],
       [77, 77, 77, ..., 77, 77, 77],
       [77, 77, 77, ..., 77, 77, 77]], dtype=int32)

In [7]:
dtmp.shape

(2601, 1, 224, 224)

In [5]:
shape_df = make_shape_labels(image_dir)
image_data = load_images(shape_df, debug=True)


In [6]:
image_data.shape

(2601, 1, 224, 224)

In [10]:


fn2save

'/usr/local/serenceslab/maggie/shapeDim/Analysis/image_similarity/features/Images_grid3_all.h5py'

In [28]:
with h5py.File(fn2save, 'w') as f:
    
    f.create_dataset("stimuli", data=image_data, dtype='i')
    f.close()

In [29]:
image_data[0,0,:,:]

array([[77., 77., 77., ..., 77., 77., 77.],
       [77., 77., 77., ..., 77., 77., 77.],
       [77., 77., 77., ..., 77., 77., 77.],
       ...,
       [77., 77., 77., ..., 77., 77., 77.],
       [77., 77., 77., ..., 77., 77., 77.],
       [77., 77., 77., ..., 77., 77., 77.]], dtype=float32)

In [3]:

def load_images(shape_df, debug=False):
    
    n_images = shape_df.shape[0]
    if debug:
        n_images_load = 10
    else:
        n_images_load = n_images
        
    first_image = PIL.Image.open(shape_df['filename_full'][0])
    n_pix = first_image.size[0]
    
    # [images x color_channels x height x width]
    image_array = np.zeros((n_images,1,n_pix,n_pix),dtype=np.float32)
    
    for ii in range(n_images_load):
        
        im = PIL.Image.open(shape_df['filename_full'][ii])
        imdat = np.reshape(np.array(im.getdata()), im.size)
        
        image_array[ii,0,:,:] = imdat
        
    return image_array



def make_shape_labels(image_dir):

    ## Create a csv file that labels all the images in dataset
    # only need to run this once to make the file (saving to same folder where images are)

    # create image labels
    start=0; stop=5; step=0.1
    grid_x = np.round(np.arange(start,stop+step,step),1)
    grid_y = np.round(np.arange(start,stop+step,step),1)
    x, y = np.meshgrid(grid_x, grid_y)
    all_grid_points = np.column_stack((x.ravel(),y.ravel()))

    center = 2.5  # center of shape space is the "boundary"
    all_quadrant = np.zeros([np.shape(all_grid_points)[0],1]);
    all_quadrant[np.logical_and(all_grid_points[:,0]>center, all_grid_points[:,1]>center)] = 1;
    all_quadrant[np.logical_and(all_grid_points[:,0]<center, all_grid_points[:,1]>center)] = 2;
    all_quadrant[np.logical_and(all_grid_points[:,0]<center, all_grid_points[:,1]<center)] = 3;
    all_quadrant[np.logical_and(all_grid_points[:,0]>center, all_grid_points[:,1]<center)] = 4;

    labels_task1 = np.zeros(np.shape(all_quadrant))
    labels_task1[np.isin(all_quadrant,[1,4])] = 2
    labels_task1[np.isin(all_quadrant,[2,3])] = 1

    labels_task2 = np.zeros(np.shape(all_quadrant))
    labels_task2[np.isin(all_quadrant,[1,2])] = 2
    labels_task2[np.isin(all_quadrant,[3,4])] = 1

    labels_task3 = np.zeros(np.shape(all_quadrant))
    labels_task3[np.isin(all_quadrant,[1,3])] = 2
    labels_task3[np.isin(all_quadrant,[2,4])] = 1

    filenames_full = [os.path.join(image_dir, 'Shape_%.2f_%.2f.png'%(x,y)) \
                      for x,y in zip(all_grid_points[:,0], all_grid_points[:,1])]
    
    shape_df = pd.DataFrame.from_dict({'coord_axis1':all_grid_points[:,0], 'coord_axis2':all_grid_points[:,1],
     'quadrant':np.squeeze(all_quadrant), 'labels_task1':np.squeeze(labels_task1),
     'labels_task2':np.squeeze(labels_task2), 'labels_task3':np.squeeze(labels_task3),
     'filename_full': filenames_full})

    return shape_df
  

In [ ]:
from 

In [ ]:
shape_df = make_shape_labels(image_dir)
fn2save = os.path.join(feat_dir, 'Image_labels_grid3.csv')
shape_df.to_csv(fn2save)

image_data = load_images(shape_df, debug=debug)
